In [1]:
from manim import *
import jupyter_capture_output
from numpy import linalg as npl

video_scene = " -v WARNING --disable_caching brown_pot_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s brown_pot_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [2]:
# potential function
def V(x, y):
    # return -1 / np.sqrt(x**2 + y**2 + 1)
    return x**2 + y**2


# Langevin differential equation of Brownian motion in potential: takes state of the system and returns its derivative array
def langevin_brown_DEQ(state, params):
    # system state
    x = state[0]
    y = state[1]
    v_x = state[2]
    v_y = state[3]
    # force calculation
    gamma = params[0]
    sigma = params[1]
    angle = np.random.uniform(0, 2*np.pi)
    delta = 10e-4
    a_x = -gamma * v_x - (V(x+delta, y) - V(x-delta, y)) / (2*delta) + sigma*np.cos(angle)
    a_y = -gamma * v_y - (V(x, y+delta) - V(x, y-delta)) / (2*delta) + sigma*np.sin(angle)
    # derivative array calculation
    d_state = np.zeros_like(state)
    d_state[0] = v_x
    d_state[1] = v_y
    d_state[2] = a_x
    d_state[3] = a_y
    return d_state


# verlet integrator
def verlet_step(state, delta_t, params):
    N = int(len(state) / 2)
    # calculation support a1
    a1 = langevin_brown_DEQ(state, params)
    # using a1: recalculation first half of state array
    for i in range(N):
        state[i] += a1[i] * delta_t + a1[N+i] * delta_t**2/2
    # calculation support a2
    a2 = langevin_brown_DEQ(state, params)
    # using a2: recalculation second half of state array
    for i in range(N):
        state[N+i] += (a1[N+i] + a2[N+1]) * delta_t/2
    return state


# calculation of mean square deviation
def get_msd(list_of_states):
    N = len(list_of_states)
    r_sum = 0
    for state in list_of_states:
        x = state[0]
        y = state[1]
        r_sum += x**2 + y**2
    return r_sum / N

In [3]:
%%capture_video --path "animations/brownian_motion/Brownian_Motion_Potential.mp4"
%%manim -qm --fps 60 $video_scene

class brown_pot_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        headline = Title("Brownian Motion in a Confining Potential", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)  
        self.add(headline)


        # parameters
        origin = np.array([-3, -1, 0])
        gamma = 1
        sigma = 10
        params = (gamma, sigma)
        delta_t = 10e-3

        # seed !!!
        np.random.seed(3)


        # add particles
        brown_1 = Circle(radius = 0.1, color = WHITE, fill_color = WHITE, fill_opacity = 0.5).move_to(origin)
        brown_2 = Circle(radius = 0.1, color = RED, fill_color = RED, fill_opacity = 0.5).move_to(origin)
        brown_3 = Circle(radius = 0.1, color = BLUE, fill_color = BLUE, fill_opacity = 0.5).move_to(origin)

        brown_1.state = np.zeros((4, ))
        brown_2.state = np.zeros((4, ))
        brown_3.state = np.zeros((4, ))

        brown_1.color = WHITE
        brown_2.color = RED
        brown_3.color = BLUE

        self.add(brown_1)
        self.add(brown_2)
        self.add(brown_3)


        # coordinate system for MSD over time
        axis_center = np.array([3, -0.5, 0])
        x_range = [0, 30, 5]
        y_range = [0, 7, 1]
        axis = Axes(x_range = x_range, y_range = y_range, x_length = 4, y_length = 4, axis_config = {'color': WHITE, "tip_width": 0.15, "tip_height": 0.15}).move_to(axis_center)
        ax_xlabel = axis.get_x_axis_label(Tex(r"$t$", font_size = 36, color = WHITE))#.shift(0.1 *LEFT)
        ax_ylabel = axis.get_y_axis_label(Tex(r"$<r^2>$", font_size = 36, color = WHITE))#.shift(0.15 * DOWN)
        plot_line = Line(start = axis.c2p(0, 0), end = axis.c2p(0, 0), color = WHITE, stroke_width = 4, stroke_opacity = 0.75)
        plot_line.time = 0
        plot_line.msd = get_msd([brown_1.state, brown_2.state, brown_3.state])
        self.add(axis, ax_xlabel, ax_ylabel, plot_line)



        # particle updater
        def particle_updater(particle):
            particle_state = particle.state.copy()
            new_particle_state = verlet_step(particle.state, delta_t, params)
            # connecting old and new position
            particle_connector = Line(start = np.array([*particle_state[0:2], 0]) + origin, end = np.array([*new_particle_state[0:2], 0]) + origin, color = particle.color, stroke_opacity = 0.5)
            self.add(particle_connector)
            # update the state of the particle to its new position
            particle.state = new_particle_state
            # move particle to new location
            particle.move_to(np.array([*new_particle_state[0:2], 0]) + origin)


        # plot updater
        def plot_updater(plot_line):
            time_old = plot_line.time
            time_new = timeline.get_value()
            msd_old = plot_line.msd
            msd_new = get_msd([brown_1.state, brown_2.state, brown_3.state])

            plot_line.msd = msd_new
            plot_line.time = time_new

            # print(time_old, time_new, msd_old, msd_new)
            new_plot_line = Line(start = axis.c2p(time_old, msd_old), end = axis.c2p(time_new, msd_new), color = WHITE, stroke_width = 4, stroke_opacity = 0.75)
            self.add(new_plot_line)


        # print(brown_1.state)
        # animation
        brown_1.add_updater(particle_updater)
        brown_2.add_updater(particle_updater)
        brown_3.add_updater(particle_updater)
        plot_line.add_updater(plot_updater)

        timeline = ValueTracker(0)
        self.play(timeline.animate.set_value(30), rate_func= linear, run_time = 30)
        self.wait(5)   

Output saved by overwring previous file at animations/brownian_motion/Brownian_Motion_Potential.mp4.
